In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint 
import plotly.express as px
import plotly.graph_objects as go 
import plotly.graph_objects as px
from plotly.subplots import make_subplots
import requests
from pprint import pprint 
import configparser
from urllib.parse import urljoin 
from datetime import datetime, timedelta

## Functions

In [ ]:
INTERVAL = 1500
MAX_DATA = 100000 

def two_bars(x, y1, y2, name):
    fig = px.Figure(data=[
        go.Bar(x=x, y=y1, yaxis='y', offsetgroup=1),
        go.Bar(x=x, y=y2, yaxis='y2', offsetgroup=2)
        # go.Bar(x=df_coinbase.datetime[:50], y=df_coinbase.coinbase_premium_gap[:50], yaxis='y', offsetgroup=1, name='premium'),
        # go.Bar(x=df_coinbase.datetime[:50], y=df_price.price_usd_close[:50], yaxis='y2', offsetgroup=2, name='price')
    ],
    
    layout={
        'yaxis': {'title': name[0]},
        'yaxis2': {'title': name[1], 'overlaying':'y', 'side': 'right'},
    })
    
    fig.update_layout(height=1000, width=4000, barmode='group', yaxis=dict(range=[y1.min(), y1.max()]), yaxis2=dict(range=[y2.min(), y2.max()]))       
    fig.show()


def get_one_day_data(db, previous_n):
    # assume that db[0] is the latest data... 
    interval = INTERVAL
    return db[interval*previous_n: interval*(previous_n+1)]


def two_lines_parallel(x, y1, y2, name):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)
    fig.add_trace(go.Scatter(x=x, y=y1, yaxis='y'), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=y2, yaxis='y2'), row=2, col=1)
    
    fig.update_layout(height=1000, width=4000, yaxis=dict(range=[y1.min(), y1.max()]), yaxis2=dict(range=[y2.min(), y2.max()]))       
    fig.show()


def two_bars_parallel(x, y1, y2, name):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)
    fig.add_trace(go.Bar(x=x, y=y1, yaxis='y', name=name[0]), row=1, col=1)
    fig.add_trace(go.Bar(x=x, y=y2, yaxis='y2', name=name[1]), row=2, col=1)
    
    fig.update_layout(height=1000, width=4000, yaxis=dict(range=[y1.min(), y1.max()]), yaxis2=dict(range=[y2.min(), y2.max()]))       
    fig.show()



In [ ]:
def simulation(signals, prices, long_condition, short_condition):
    # signals: datetime, signal format 

    cost = 0 
    unrealized_profit = 0
    profit = 0 
    fee = 0 
    position = 0 

    for signal in signals:
        if long_condition(signal):
            price = get_price(signal.datetime)
            cost += price + fee 
            position += 1
        else if short_condition(signal): 
            price = get_price(signal.datetime)
            cost += price + fee 








    return True


In [ ]:
make_subplots?

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['CRYPTOQUANT']['API_KEY']
API_URL = config['CRYPTOQUANT']['API_URL']

headers = {'Authorization': 'Bearer ' + API_KEY}

In [ ]:
PATH = 'btc/market-data/coinbase-premium-index'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'min',
    'limit': MAX_DATA
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
df_coinbase = pd.DataFrame(response['result']['data'])

in_date = df_coinbase.loc[0].datetime
d = datetime.strptime(in_date, '%Y-%m-%d %H:%M:%S')
new_date = d.strftime('%Y%m%dT%H%M%S')

PATH = 'btc/market-data/price-usd'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'min',
    'to': new_date,
    'limit': MAX_DATA
}

response = requests.get(URL, headers=headers, params=params).json()
df_price = pd.DataFrame(response['result']['data'])


In [ ]:
df_coinbase_0 = get_one_day_data(df_coinbase, 0)
df_price_0 = get_one_day_data(df_price, 0)

In [ ]:
df_price_0['datetime'] = pd.to_datetime(df_price_0['datetime'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df_price_0

type(df_price_0.loc[0, 'datetime'])

In [ ]:
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

def find_date(x, yesteday):
    d = datetime.strptime(x.datetime, '%Y-%m-%d %H:%M:%S')
    if (d.strftime('%Y-%m-%d') == yesterday): 
        return True 
    return False
how_many_data_one_day = df_coinbase_0.apply(lambda x: find_date(x, yesterday), axis=1)
how_many_data_one_day.value_counts()

In [ ]:
df_coinbase_0 = get_one_day_data(df_coinbase, 0)
df_price_0 = get_one_day_data(df_price, 0)
two_bars(x=df_coinbase_0.datetime, y1=df_coinbase_0.coinbase_premium_gap, y2=df_price_0.price_usd_close, name=['premium', 'price'])


In [ ]:
two_bars_parallel(x=df_coinbase_0.datetime, y1=df_coinbase_0.coinbase_premium_gap, y2=df_price_0.price_usd_close, name=['premium', 'price'])

In [ ]:
df_coinbase_0

In [ ]:
df_price_0.datetime

In [ ]:
df_price_0[df_price_0.datetime.strftime(%Y%m%d) == '2021']

In [ ]:
one_day = get_one_day_data(df_coinbase, 1)
one_day


In [ ]:
one_day = 1500





df_coinbase_1 = df_coinbase[one_day+1:one_day*2]
df_price_1 = df_price[one_day+1:one_day*2]
pprint(df_coinbase[:one_day])
pprint(df_price[:one_day])

In [ ]:
two_bar_graphs(x=df_coinbase_1.datetime, y1=df_coinbase_1.coinbase_premium_gap, y2=df_price_1.price_usd_close, name=['premium', 'price'])


In [ ]:
one_day = 1500

df_coinbase_1 = df_coinbase[:one_day]
df_price_1 = df_price[:one_day]
pprint(df_coinbase[:one_day])
pprint(df_price[:one_day])

In [ ]:
two_bar_graphs(x=df_coinbase_1.datetime, y1=df_coinbase_1.coinbase_premium_gap, y2=df_price_1.price_usd_close, name=['premium', 'price'])


In [ ]:
one_day = 1500

df_coinbase_1 = df_coinbase[2*one_day+1 :one_day*3]
df_price_1 = df_price[2*one_day+1 :one_day*3]
two_bar_graphs(x=df_coinbase_1.datetime, y1=df_coinbase_1.coinbase_premium_gap, y2=df_price_1.price_usd_close, name=['premium', 'price'])


In [ ]:
one_day = 1500

df_coinbase_1 = df_coinbase[3*one_day+1 :one_day*4]
df_price_1 = df_price[3*one_day+1 :one_day*4]
two_bar_graphs(x=df_coinbase_1.datetime, y1=df_coinbase_1.coinbase_premium_gap, y2=df_price_1.price_usd_close, name=['premium', 'price'])